### The problem has >1000 classes (types of clothing), and each data point can have multiple labels. Therefore this is a multilabel, multiclass problem. A potential solution is to use a OneVsRestClassifier. However, because of the scale of the problem, and my personal limitation to a single laptop for computational power, this algorithm cannot be tested for this data challenge.

In [1]:
import numpy
import seaborn as sns
import pandas as pd
import matplotlib
from matplotlib import *
%matplotlib inline

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


#### Get the labels and the product datasets. The simplest way of creating features from the text (which contains > 6 languages), is to put it into a 'bag of words'. The TfidfVectorizer also weights the words based on their frequency (although this is likely to not work as well since the words are in so many different languages). Because of the size of the dataset and the limitations of memory on my laptop, I cannot make a bag of words out of the descriptions.

In [2]:
in_labels = pd.read_csv('data/input-data-labels.csv',encoding='utf-8')
in_labelling = pd.read_csv('data/input-data-labeling.csv',encoding='utf-8')

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
vectorizer = TfidfVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 1000) 

In [5]:
name_features = vectorizer.fit_transform(in_labelling['name'])
name_features = name_features.toarray()

/usr/local/lib/python2.7/dist-packages/numpy/core/fromnumeric.py:2652: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)


#### Select the features I want to keep (my bag of words and the shop, brand and price) and store them in a single dataframe. Format the features so they are all numerical (i.e. change the shop and brand to a list of numbers). 

In [6]:
name_df = pd.DataFrame(name_features)

In [7]:
del in_labelling['name']
del in_labelling['description']
del in_labelling['id']

In [8]:
label_targets = in_labelling['labels']

In [9]:
del in_labelling['labels']

In [10]:
from sklearn import preprocessing
# Discreet value integer encoder
label_encoder = preprocessing.LabelEncoder()

In [11]:
in_labelling['shop'] = label_encoder.fit_transform(in_labelling['shop'])
in_labelling['brand'] = label_encoder.fit_transform(in_labelling['brand'])

In [12]:
feature_df = pd.concat([in_labelling,name_df],axis=1)

#### Format the labels as lists of numerical values to pass to the OneVsRestClassifier

In [13]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn import cross_validation
from sklearn import ensemble

In [14]:
def get_labels(df_text):
    split_text = df_text.split(',')
    return split_text

In [15]:
label_binarizer = preprocessing.LabelBinarizer()
label_targets2 = label_targets.apply(get_labels)
label_targets2 = label_targets2.as_matrix()

In [16]:
print label_targets2[0]

[u'Bekleidung', u'Hemden', u'Business Hemden', u'Haifischkragen Hemden', u'Langarm Hemden', u'Smokinghemden']


In [17]:
label_targets2 = label_binarizer.fit_transform(label_targets2)

In [18]:
print numpy.count_nonzero(label_targets2)

1309532


#### Attempt to run the classifier. Failed due to lack of memory on my laptop

In [ ]:
from sklearn.cross_validation import train_test_split
X_train,X_test,y_train,y_test = train_test_split(feature_df,label_targets2,test_size=0.95)

In [ ]:
clf = OneVsRestClassifier(ensemble.RandomForestClassifier()).fit(X_train, y_train)

/usr/lib/python2.7/dist-packages/sklearn/multiclass.py:57: UserWarning: Label not 2 is present in all training examples.
  str(classes[c]))
/usr/lib/python2.7/dist-packages/sklearn/multiclass.py:57: UserWarning: Label not 13 is present in all training examples.
  str(classes[c]))
/usr/lib/python2.7/dist-packages/sklearn/multiclass.py:57: UserWarning: Label not 19 is present in all training examples.
  str(classes[c]))
/usr/lib/python2.7/dist-packages/sklearn/multiclass.py:57: UserWarning: Label not 33 is present in all training examples.
  str(classes[c]))
/usr/lib/python2.7/dist-packages/sklearn/multiclass.py:57: UserWarning: Label not 40 is present in all training examples.
  str(classes[c]))
/usr/lib/python2.7/dist-packages/sklearn/multiclass.py:57: UserWarning: Label not 41 is present in all training examples.
  str(classes[c]))
/usr/lib/python2.7/dist-packages/sklearn/multiclass.py:57: UserWarning: Label not 42 is present in all training examples.
  str(classes[c]))
/usr/lib/pytho

In [ ]:
y_preds = clf.predict(X_test)

In [ ]:
print y_preds[:50],y_test[:50]